# 합성곱 신경망

컨브넷의 정의 알아보기. 

완전연결 네트워크와의 성능비교.

-- 기존 2장에서 실습한 MNIST  예제를 컨브넷으로 학습하고 비교.


컨브넷은 (image_height, image_width, image_channels) 크기의 입력 텐서를 사용. (배치 차원은 미포함)

아래는 기본적인 컨브넷의 형태

In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


모델 구조 확인

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D와 MaxPooling2D 층의 출력 : (height, width, channels) 크기의 3D 텐서  

높이와 넓이 차원은 네트워크가 깊어질수록 작아지는 경향이 있음. 

채널 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절(32개 또는 64개)

다음 단계에서 **마지막 층**의 ((3, 3, 64) 크기인) 출력 텐서를 **완전 연결 네트워크** 주입.  (Dense 층을 쌓은 분류기)

이 분류기(마지막 층)는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서. 

그래서 먼저 3D 출력을 1D 텐서로 펼친 다음 몇 개의 Dense 층을 추가  

**10개의 클래스**를 분류하기위해 마지막에 출력크기를 10으로 지정.

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

모델 구조 확인

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

MNIST 숫자 이미지를 컨브넷을 통해 훈련. (2장 코드 재사용)

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 8s 130us/step - loss: 0.1691 - acc: 0.9476
Epoch 2/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0474 - acc: 0.9857
Epoch 3/5
60000/60000 [==============================] - 4s 60us/step - loss: 0.0326 - acc: 0.9898
Epoch 4/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0246 - acc: 0.9922
Epoch 5/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0196 - acc: 0.9941


테스트 데이터 모델 평가

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("테스트 loss")
print(test_loss)
print("테스트 acc")
print(test_acc)

10000/10000 [==============================] - 0s 40us/step
테스트 loss
0.025002007406899066
테스트 acc
0.9925


2장의 완전연결 네트워크 acc : 97.8%
 기본  컨브넷  네트워크 acc : 99.2%
 
 에러율도 많이 줄었음.